# Данные

In [2]:
import pandas
tweets = pandas.DataFrame.from_csv("Tweets data.csv")

In [3]:
print(tweets[::5000])

        Sentiment                                      SentimentText
ItemID                                                              
1               0                       is so sad for my APL frie...
5002            1       very very very happy and slightly skinny  xx
10012           1  &quot;I feel better now, better than James Bro...
15012           1  ...what can I say ....what a surprise...http:/...
20012           0                                     @ ponyp: hey! 
25012           1       @404world yes I'm going to find one..pronto 
30012           0  @AdamJonesey mmm id love some parma violets ri...
35012           1  @AlTheYid Roast beef and wasabi... yummmmmmmmm...
40012           0  @amandafortier ahhh. yeah. kinda. I tweet from...
45012           0                 @antdeshawn ... the pain is worst 
50012           1  @Ashtarte Just be your usual supportive self. ...
55012           0  @azizabatini86 and that's fine--u kno, the dry...
60012           0       @Bellemord

In [4]:
# print(tweets[:10])
for i in tweets.Sentiment[:10]:
    print(i)

0
0
1
0
0
0
1
0
1
1


# preprocessing
Извлечём из твитов признаки

In [5]:
texts = tweets["SentimentText"]
Y = tweets["Sentiment"]

In [6]:
def extract_features(text):
    features = []
    
    
    text = text.lower()
    
    #<для начала будем использовать частоты букв как признаки>
    letters = "abcdefghijklmnopqrstuvwxyz"    
    for l in letters:
        features.append(text.count(l))
    
    return features

In [7]:
extract_features ('AAaab')

[4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [8]:
import numpy
X = []
for text in texts:
    X.append(extract_features(text))
X = numpy.array(X)


# Fit a model
Научим на наших признаках простую линейную модель

In [9]:
#порежем данные
X_train = X[:70000]
Y_train = Y[:70000]
X_test = X[70000:]
Y_test = Y[70000:]

In [10]:
from sklearn.linear_model import LogisticRegression

# model = <обучите логистическую регрессию>
model = LogisticRegression()

In [11]:
print (model)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


In [12]:
y = model.fit(X_train,Y_train)

In [13]:
model.coef_

array([[ 0.03976119, -0.02395792,  0.03336063, -0.07673787,  0.01470141,
         0.04481567,  0.04607898,  0.00136932, -0.04444427,  0.14625134,
         0.05731417,  0.04799672, -0.05269558, -0.03541793,  0.01029763,
         0.01853502,  0.23346672,  0.02136664, -0.06216603, -0.03674036,
         0.03427765,  0.02921815, -0.01795578,  0.06048928,  0.04883713,
         0.06188793]])

In [14]:
model.fit(X[7000:14000], Y[7000:14000])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [15]:
model.coef_

array([[ 0.01513508,  0.0253065 ,  0.01467999, -0.06508438, -0.02102025,
         0.07285677,  0.01079471, -0.00807934, -0.055264  ,  0.23026256,
         0.0582649 ,  0.03025168,  0.0808289 , -0.03890496, -0.00357455,
         0.00564272,  0.35088587,  0.00517337, -0.01687005, -0.00699748,
        -0.01350235,  0.09033838,  0.02882315,  0.15865924,  0.07377224,
         0.23060619]])

In [16]:
y.coef_

array([[ 0.01513508,  0.0253065 ,  0.01467999, -0.06508438, -0.02102025,
         0.07285677,  0.01079471, -0.00807934, -0.055264  ,  0.23026256,
         0.0582649 ,  0.03025168,  0.0808289 , -0.03890496, -0.00357455,
         0.00564272,  0.35088587,  0.00517337, -0.01687005, -0.00699748,
        -0.01350235,  0.09033838,  0.02882315,  0.15865924,  0.07377224,
         0.23060619]])

In [17]:
from sklearn.metrics import roc_auc_score

In [18]:
model.predict_proba()

TypeError: predict_proba() takes exactly 2 arguments (1 given)

In [19]:
Y_pred_train = model.predict_proba(X_train)[:,1]
Y_pred_test = model.predict_proba(X_test)[:,1]
print(u"Обучение:") 
print(roc_auc_score(Y_train,Y_pred_train))
#recieve operator curve area under curve
print(u"Тест:") 
print(roc_auc_score(Y_test,Y_pred_test))


Обучение:
0.567601786052
Тест:
0.558523325999


# opinion mining

Давайте использовуем нашу модель чтобы понять, что людям сейчас нравится/не нравится

In [20]:
import numpy
with open("testdata.txt") as fin:
    test_tweets = fin.read()[:-1].split('\n')
test_tweets = numpy.array(test_tweets)

In [21]:
print test_tweets[:5]

['" I don\'t care what anyone says, I like Hillary Clinton.'
 'have an awesome time at purdue!..'
 "Yep, I'm still in London, which is pretty awesome: P Remind me to post the million and one pictures that I took when I get back to Markham!..."
 "Have to say, I hate Paris Hilton's behavior but I do think she's kinda cute.."
 'i will love the lakers.']


In [22]:
for i in test_tweets[:1]:
    print (i)

" I don't care what anyone says, I like Hillary Clinton.


In [23]:
X_ops = []
for tweet in test_tweets:
    X_ops.append(extract_features(tweet))
X_ops = numpy.array(X_ops)


In [24]:
sentiments = <предскажите эмоциональную окраску x_ops>
sentiment_order = numpy.argsort(-sentiments)

SyntaxError: invalid syntax (<ipython-input-24-37fcba1343d9>, line 1)

In [25]:
### теперь посмотрим на наиболее любимые/ненавистные людям вещи

In [26]:
best_ids = sentiment_order[:10]

for tweet, score in zip(test_tweets[best_ids], sentiments[best_ids]):
    print score,':',tweet

NameError: name 'sentiment_order' is not defined

In [27]:
worst_ids = sentiment_order[-10:]

for tweet, score in zip(test_tweets[worst_ids], sentiments[worst_ids]):
    print score,':',tweet

NameError: name 'sentiment_order' is not defined

# bag of words

In [28]:
import nltk
stemmer = nltk.snowball.RussianStemmer()
re_tokenizer = nltk.tokenize.RegexpTokenizer(r'[А-Яа-яA-Za-z0-9\']+')
text_words=[]
for i in test_tweets: 
    temp=re_tokenizer.tokenize(i)
    for y in temp:
        text_words.append (y)

In [29]:
print (text_words[:5])

['I', "don't", 'care', 'what', 'anyone']


In [36]:
from collections import Counter
counters = Counter(text_words)

In [37]:
l=0
for i in counters:
    print i
    print counters[i]
    l+=1
    if l>10: break

raining
4
Uni
1
yellow
6
CBR600RR
1
four
11
prices
4
Does
4
Olympics
1
friend's
1
hanging
5
centimeter
1


In [41]:
words = [w for w, cnt in counters.most_common(100)]
word_to_id = {word:i for i,word in enumerate(list(words))}

In [42]:
l=0
for i in words:
    print i
    l+=1
    if l>5: break
        
l=0
for i in word_to_id:
    print i
    print word_to_id[i]
    l+=1
    if l>5: break

I
the
i
and
to
love
all
83
just
88
Paris
47
london
99
go
92
still
66


In [43]:
print(len(word_to_id))

100


In [ ]:
#выделите 1000 cлов, которые имеют наиболее разную вероятность войти в позитивный и негативный твит
bag_of_words = <слова-признаки>

In [44]:
def extract_features(text):
    features = numpy.zeros(100)
    
    words=re_tokenizer.tokenize(text)
    
    
    for i in word_to_id.keys():
        if i in words:    
            features[word_to_id[i]]=1
        else:
            features[word_to_id[i]]=0
    
    return numpy.array(features)

In [47]:
import numpy
X = []
for text in texts:
    X.append(extract_features(text))
X = numpy.array(X)


In [46]:
from sklearn.linear_model import LogisticRegression


modelW = LogisticRegression()



In [48]:
modelW.fit(X[:70000], Y[:70000])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [49]:

X_train = X[:70000]
Y_train = Y[:70000]
X_test = X[70000:]
Y_test = Y[70000:]

In [50]:
modelW.coef_

array([[-0.48043097,  0.1307492 , -0.6869916 ,  0.06659804, -0.08715588,
         1.05551664, -0.09387772,  0.20143918, -0.31857099,  0.01000838,
        -0.09726974,  0.04799031, -1.51054736, -0.41014664,  0.13781958,
        -0.21389823, -0.22958884,  0.25187863, -1.1159351 ,  0.07265507,
        -0.22837775,  0.68373938,  1.1564716 , -0.06377541,  0.05292679,
        -0.21719257, -0.20272343, -2.55011768, -0.64357111,  0.26707048,
         0.04686916, -0.42564515,  0.23137983,  0.59255256, -0.62076144,
         0.11536619,  0.13215331,  0.60576724, -0.53214702,  0.1193036 ,
         0.53850964,  0.55689599,  0.24486757,  0.01599564, -0.13809072,
        -0.1016285 , -0.10636934,  0.08281296,  0.        , -0.80758178,
        -0.2016889 , -0.02871098,  0.01478995,  0.27889685, -0.10749062,
        -0.67504535,  0.4831555 ,  0.00732276, -0.02257154,  1.04127425,
        -1.6226956 , -0.37911592, -0.22720661, -0.15834625, -0.17046448,
        -0.42313468, -0.55408508,  0.21777448,  0.3

In [51]:
Y_pred_train = modelW.predict_proba(X_train)[:,1]
Y_pred_test = modelW.predict_proba(X_test)[:,1]
print(u"Обучение:") 
print(roc_auc_score(Y_train,Y_pred_train))
#recieve operator curve area under curve
print(u"Тест:") 
print(roc_auc_score(Y_test,Y_pred_test))

Обучение:
0.684341336656
Тест:
0.685823367813


In [ ]:
Y_pred_train = <предскажите вероятности для обучающей выборки copy-paste>
Y_pred_test = <предскажите вероятности для обучающей выборки copy-paste>
print("Обучение:",roc_auc_score(Y_train,Y_pred_train))
print("Тест:",roc_auc_score(Y_test,Y_pred_test))


# collocations
Найдём наиболее частые коллокации и используем их как признаки

In [52]:
import nltk
from nltk import collocations

In [66]:
text_words= [i.lower() for i in text_words]

In [67]:
text_words

['i',
 "don't",
 'care',
 'what',
 'anyone',
 'says',
 'i',
 'like',
 'hillary',
 'clinton',
 'have',
 'an',
 'awesome',
 'time',
 'at',
 'purdue',
 'yep',
 "i'm",
 'still',
 'in',
 'london',
 'which',
 'is',
 'pretty',
 'awesome',
 'p',
 'remind',
 'me',
 'to',
 'post',
 'the',
 'million',
 'and',
 'one',
 'pictures',
 'that',
 'i',
 'took',
 'when',
 'i',
 'get',
 'back',
 'to',
 'markham',
 'have',
 'to',
 'say',
 'i',
 'hate',
 'paris',
 "hilton's",
 'behavior',
 'but',
 'i',
 'do',
 'think',
 "she's",
 'kinda',
 'cute',
 'i',
 'will',
 'love',
 'the',
 'lakers',
 "i'm",
 'so',
 'glad',
 'i',
 'love',
 'paris',
 'hilton',
 'too',
 'or',
 'this',
 'would',
 'be',
 'excruciating',
 'considering',
 'most',
 'geico',
 'commericals',
 'are',
 'stupid',
 'i',
 'liked',
 'mit',
 'though',
 'esp',
 'their',
 'little',
 'info',
 'book',
 'before',
 'i',
 'left',
 'missouri',
 'i',
 'thought',
 'london',
 'was',
 'going',
 'to',
 'be',
 'so',
 'good',
 'and',
 'cool',
 'and',
 'fun',
 'and',

In [68]:
cfinder = collocations.BigramCollocationFinder.from_words(text_words)
cfinder.apply_freq_filter(5)
measure = collocations.BigramAssocMeasures.raw_freq

In [69]:
print(cfinder.nbest(measure,100))

[('i', 'love'), ('i', 'hate'), ('paris', 'hilton'), ('tom', 'cruise'), ('san', 'francisco'), ('angelina', 'jolie'), ('and', 'i'), ('love', 'the'), ('the', 'lakers'), ('i', 'like'), ('love', 'my'), ('i', 'think'), ('but', 'i'), ('hate', 'paris'), ('that', 'i'), ('to', 'be'), ('hate', 'tom'), ('i', 'want'), ('i', 'miss'), ('i', 'need'), ('so', 'much'), ('love', 'paris'), ('of', 'the'), ('seattle', 'sucks'), ('i', 'really'), ('jolie', 'is'), ('stupid', 'ucla'), ('sucks', 'i'), ('is', 'beautiful'), ('by', 'the'), ('in', 'the'), ('hate', 'london'), ('is', 'awesome'), ('i', 'still'), ('love', 'angelina'), ('to', 'go'), ('as', 'i'), ('i', 'have'), ('is', 'a'), ('i', 'can'), ('in', 'my'), ('in', 'seattle'), ('to', 'see'), ('cruise', 'i'), ('to', 'the'), ('hate', 'the'), ('hilton', 'i'), ('stupid', 'lakers'), ('of', 'that'), ('love', 'ucla'), ('which', 'is'), ('is', 'so'), ('awesome', 'i'), ('want', 'a'), ('my', 'macbook'), ('love', 'seattle'), ('love', 'shanghai'), ('the', 'way'), ('like', 'th

In [ ]:
def extract_features(text):
    features = []
    
    <bag of words для коллокаций>
    
    return features

In [ ]:
import numpy
X = []
for text in texts:
    X.append(extract_features(text))
X = numpy.array(X)


In [ ]:
from sklearn.linear_model import LogisticRegression

model = <обучите модель>

In [ ]:
Y_pred_train = <предскажите вероятности для обучающей выборки copy-paste>
Y_pred_test = <предскажите вероятности для обучающей выборки copy-paste>
print("Обучение:",roc_auc_score(Y_train,Y_pred_train))
print("Тест:",roc_auc_score(Y_test,Y_pred_test))
